In [3]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision import datasets
import torchvision
import matplotlib.pyplot as plt
import cv2
from pathlib import Path
import random
import shutil

In [4]:
print(f'cwd: {Path.cwd()}')

def split_folder(folder):
    all_files = Path(f'{folder}/').glob(f'*.png')
    all_files = list(all_files)
    print(list(all_files)[0:3])
    random.shuffle(all_files)
    split = 0.7
    split_index = int(np.floor(len(all_files) * split))
    training = all_files[0:split_index]
    testing = all_files[split_index:len(all_files)]
    return training, testing

Path.mkdir(Path('./train'), exist_ok=True)
Path.mkdir(Path('./validate'), exist_ok=True)

# data = {}
# data[shape] = split_folder(shape)
for shape in ['circle', 'star', 'triangle', 'square']:
    print(f'shape: {shape}')
    for dataset in ['train', 'validate']:
        Path.mkdir(Path(f'./{dataset}/{shape}'), exist_ok=True)
    train, validate = split_folder(shape)
    for file in train:
        shutil.copy(file, f'./train/{shape}')
    for file in validate:
        shutil.copy(file, f'./validate/{shape}')

cwd: c:\Users\sylvi\Documents\GitKraken\python-tests\ML\Shape Detection
shape: circle
[WindowsPath('circle/0.png'), WindowsPath('circle/1.png'), WindowsPath('circle/10.png')]
shape: star
[WindowsPath('star/0.png'), WindowsPath('star/1.png'), WindowsPath('star/10.png')]
shape: triangle
[WindowsPath('triangle/0.png'), WindowsPath('triangle/1.png'), WindowsPath('triangle/10.png')]
shape: square
[WindowsPath('square/0.png'), WindowsPath('square/1.png'), WindowsPath('square/10.png')]


In [11]:
image_files = datasets.ImageFolder('train', transform=transforms.ToTensor())
data_loaders = torch.utils.data.DataLoader(image_files, batch_size=5, shuffle=True)
class_name = image_files.classes
print(class_name)

['circle', 'square', 'star', 'triangle']


In [19]:
# Loading the image and labels
data_dir = './'
# Dictionary containing image info in both train and test
image_files = {x:datasets.ImageFolder(os.path.join(data_dir, x), transform=transforms.ToTensor())
                for x in ["train"]} #only using tensor transform for now
# Dictionary containing the data loader
data_loaders = {x:torch.utils.data.DataLoader(image_files[x], batch_size=5, shuffle=True) for x in 
               ["train"]}
# Getting the class names
class_name = image_files["train"].classes

In [27]:
# Building a simple CNN
class CNN(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.cnn1 = nn.Conv2d(in_channels=3, out_channels=8, kernel_size=3, 
                             stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(8)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=2)
        
        self.cnn2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=5, 
                              stride=1, padding=2)
        self.bn2 = nn.BatchNorm2d(16)
        
        self.cnn3 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5,
                             stride=1, padding=2)
        self.bn3 = nn.BatchNorm2d(32)
        
        self.fc1 = nn.Linear(1568, 600)
        self.dropout = nn.Dropout(p=0.5)
        self.fc2 = nn.Linear(600, 3)
        
    def forward(self, x):
        out = self.cnn1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.maxpool(out)
        out = self.cnn2(out)
        out = self.bn2(out)
        out = self.relu(out)
        out = self.maxpool(out)
        out = self.cnn3(out)
        out = self.bn3(out)
        self.relu(out)
        out = out.view(5, len(image_files)) # batch_size = 5
        out = self.fc1(out)
        out = self.relu(out)
        out = self.dropout(out)
        out = self.fc2(out)
        
        return (out)

In [25]:
model = CNN()
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [28]:
epochs = 150
train_loss = []
train_acc = []

for epoch in range(epochs):
    
    # Training
    correct = 0
    iterations = 0
    iter_loss = 0
    
    model.train()
    for images, labels in data_loaders['train']:
        outputs = model(images)
        loss = loss_fn(outputs, labels)
        iter_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        correct += (predicted == labels).sum().item()
        iterations += 1
    
    train_loss.append(iter_loss/iterations)
    train_acc.append(100 * correct / len(image_files))
    
    print("Epoch [{}/{}], Training loss: {:.3f}, Training accuracy: {:.3f}".format(epoch+1, epochs, train_loss[-1], train_acc[-1]))

RuntimeError: shape '[5, 400000]' is invalid for input of size 400000